<a href="https://www.kaggle.com/code/limonhalder/binary-segmentation-using-u-net?scriptVersionId=109930187" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import required libraires

# What is segmentation in deep learning?
Image result for binary segmentation in deep learning
Image segmentation is the task of clustering parts of an image together that belong to the same object class. This process is also called pixel-level classification.
In other words, it involves partitioning images (or video frames) into multiple segments or objects.In binary segmentation there exists two class of object. One object should be detect from others.

![](https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.researchgate.net%2Ffigure%2FFetal-head-segmentation-fP-CMC-segmentation-is-in-red-initial-labels-ellipses-are-in_fig7_263814805&psig=AOvVaw1YjMXz4EkN6Q5bIBfoYfLK&ust=1667564051246000&source=images&cd=vfe&ved=0CA0QjRxqFwoTCJDjy6r-kfsCFQAAAAAdAAAAABAE)

# Import required libraires

In [ ]:
import os 
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
import glob

In [ ]:
img_directory=""
mask_directory=""
size=256
num_img=999

# Preprocessing for images and mask

![](http://../input/maskimage/002_HC.png)

![](../input/maskimage/002_HC_Annotation.png)

In [ ]:
image_names=glob.glob("../input/fetal-head-circumference-data/Fetal Head Circumference/train_set/Images/*.png")

image_names.sort()

print(len(image_names))
image_names_subset=image_names[0:num_img]
images=[cv2.imread(img,0) for img in image_names_subset]
images=[cv2.resize(img,(256,256)) for img in images]
image_dataset=np.array(images)
print(image_dataset.shape)

image_dataset=np.expand_dims(image_dataset, axis=3)

In [ ]:
mask_names=glob.glob("../input/fetal-head-circumference-data/Fetal Head Circumference/train_set/Annotation/*.png")

mask_names.sort()
mask_names_subset=mask_names[0:num_img]
masks=[cv2.imread(mask,0) for mask in mask_names_subset]
masks=[cv2.resize(mask,(256,256)) for mask in masks]
mask_dataset=np.array(masks)
mask_dataset=np.expand_dims(mask_dataset, axis=3)


In [ ]:
mask_dataset.shape

In [ ]:
image_dataset=image_dataset/256
mask_dataset=mask_dataset/256



# Spliting data into train and test

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(image_dataset,mask_dataset, test_size=.2, random_state=42)

# chenity check

In [ ]:
import random
image_number=random.randint(0,len(X_train))
plt.figure(figsize=(12,6))
plt.subplot(121)
plt.imshow(X_train[image_number,:,:,0],cmap='gray')
plt.subplot(122)
plt.imshow(y_train[image_number,:,:,0], cmap="gray")
plt.show()

In [ ]:
X_train[image_number,:,:,0].shape

In [ ]:
from keras.models import Model
from keras.layers.convolutional import  Conv2D, MaxPooling2D,UpSampling2D,Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, MaxPooling2D, Concatenate,Input, BatchNormalization,Dropout,Lambda


# Convolution block with 2 conv layers and batch normalization for each layer

In [ ]:
def conv_block(input, num_filters):
    x=Conv2D(num_filters,3,padding="same")(input)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    
    
    x=Conv2D(num_filters,3,padding="same")(x)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    return x

# Encoder Block and decoder block 

In [ ]:
def encoder_block(input, num_filters):
    x=conv_block(input,num_filters)
    p=MaxPooling2D((2,2))(x)
    
    return x,p




def decoder_block(input,skip_feature, num_filters):
    x=Conv2DTranspose(num_filters,(2,2),strides=2, padding="same")(input)
    x=Concatenate()([x,skip_feature])
    x=conv_block(x,num_filters)
    return x




    

# Build U-net bloack using encoder nad decoder block

In [ ]:
def build_unet(input_shape, n_classes):
    inputs=Input(input_shape)
    
    s1,p1=encoder_block(inputs,64)
    s2,p2=encoder_block(p1,128)
    s3,p3=encoder_block(p2,256)
    s4,p4=encoder_block(p3,512)
    
    
    b1=conv_block(p4,1024) # bridge
    
    
    d1=decoder_block(b1,s4,512)
    d2=decoder_block(d1,s3,256)
    d3=decoder_block(d2, s2,128)
    d4=decoder_block(d3,s1,64)
    
    if n_classes==1:
        activation="sigmoid"
        
    else:
        activation="softmax"
    
    
    
    outputs=Conv2D(n_classes,1,padding="same",activation=activation)(d4)
    
    
    model=Model(inputs,outputs,name="U-Net")
    
    return model
    
    

In [ ]:
Img_height=image_dataset.shape[1]
Img_width=image_dataset.shape[2]
Img_channel=image_dataset.shape[3]
input_shape=(Img_height,Img_width,Img_channel)


In [ ]:
Img_height

In [ ]:
model=build_unet(input_shape,n_classes=1)
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=["accuracy"])

# History

In [ ]:
history=model.fit(X_train,y_train,
                 batch_size=3,
                 verbose=1,
                 epochs=3,
                 validation_data=(X_test,y_test),
                 shuffle=False)

In [ ]:
s=X_test[8]

In [ ]:
predicts=model.predict(X_test)


In [ ]:
thrshold=0
test_img_number=random.randint(0,len(X_test))
test_img=X_test[test_img_number]
ground_truth=y_test[test_img_number]
test_image_input=np.expand_dims(test_img,0)
predictio=(model.predict(test_image_input)[0,:,:,:]>.15).astype(np.uint8)
#predictio=(model.predict(test_image_input)[0,:,:,:]>.15)


plt.figure(figsize=(16,8))
plt.title("testing Image")
plt.subplot(221)
plt.imshow(test_img[:,:,:],cmap='gray')

plt.figure(figsize=(16,8))
plt.title("true value")
plt.subplot(222)
plt.imshow(ground_truth[:,:,:],cmap='gray')


plt.figure(figsize=(16,8))
plt.title("predicted")
plt.subplot(223)
plt.imshow(predicts[6],cmap='gray')
plt.show()

In [ ]:
predictio.shape

In [ ]:
plt.imshow(predictio,cmap='gray')

predictio and ground are in different data type. thats why can not be substract

In [ ]:
X_test.shape[0]

# Prediction for whole dataset

In [ ]:
predicts=model.predict(X_test)

In [ ]:
predicts.shape

In [ ]:
predict2=[]
masks=[]

def prediction(image_dataset):
    for i in range(image_dataset.shape[0]):
        test_img=image_dataset[i]
        ground_truth=mask_dataset[i]
        test_image_input=np.expand_dims(test_img,0)
        predict=(model.predict(test_image_input)[0,:,:,:]>.141).astype("float32")
        predict2.append(predict)
        
    return predict2    

In [ ]:
#predicts2=prediction(X_test)

In [ ]:
 predicts= np.array(predicts)

In [ ]:
predicts2.shape

In [ ]:
X_test.shape

In [ ]:
plt.subplot(122),plt.imshow(predicts[3], cmap='gray')
plt.title('mask'), plt.xticks([]), plt.yticks([])
plt.show()
plt.subplot(122),plt.imshow(y_test[73], cmap='gray')
plt.title('predict'), plt.xticks([]), plt.yticks([])
plt.show()
plt.subplot(122),plt.imshow(predicts[0], cmap='gray')
plt.title('Hough Transform'), plt.xticks([]), plt.yticks([])
plt.show()

# MSE error mesurement

In [ ]:
#img1 = cv2.imread('')
#img2 = cv2.imread(prediction)

# convert the images to grayscale
#
#
#img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
#img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
# define the function to compute MSE between two images
def mse(img1, img2):
   h, w = img1.shape
   diff = cv2.subtract(img1, img2)
   err = np.sum(diff**2)
   mse = err/(float(h*w))
   return mse



In [ ]:
ERROR1=[]
def MSE(masks, predicts):
  
    masks=[np.asarray(mask, np.float64) for mask in masks]
    masks=[img[:,:,0] for img in masks]
    

    preds=[np.asarray(pred, np.float64) for pred in predicts]
    preds=[img[:,:,0] for img in preds ]
    
    
    
    for mask , pred in zip(masks, preds):
        error=mse(mask,pred)
        ERROR1.append(error)
    return ERROR1

In [ ]:
ERROR2=[]
def MSE2(masks, predicts):
  
    masks=[np.asarray(mask, np.float64) for mask in masks]
    masks=[img[:,:,0] for img in masks]
    

    preds=[np.asarray(pred, np.float64) for pred in predicts]
    preds=[img[:,:,0] for img in preds ]
    
    
    
    for mask , pred in zip(masks, preds):
        error=mse(mask,pred)
        ERROR2.append(error)
    return ERROR2

In [ ]:
#MSE(X_test,y_test)

In [ ]:
error = MSE(y_test,predicts2)
print("Image matching Error between the two images:",error)

# Hough Transform

In [ ]:
predicts=model.predict(X_test)

In [ ]:
predicts.shape

In [ ]:
predicts2.dtype

In [ ]:
predicts=predicts*256

In [ ]:
predicts.dtype

In [ ]:
predicts=predicts.astype('uint8')

In [ ]:
predicts.dtype

In [ ]:
bgr_img=predicts[8]
plt.subplot(122),plt.imshow(bgr_img, cmap='gray')
plt.title('mask'), plt.xticks([]), plt.yticks([])
plt.show()
bgr_img=predicts2[0]
plt.subplot(121),plt.imshow(mask_dataset[9], cmap='gray')
plt.title('mask'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
bgr_img=predicts[110]

In [ ]:
if bgr_img.shape[-1] == 3:           # color image
    b,g,r = cv2.split(bgr_img)       # get b,g,r
    rgb_img = cv2.merge([r,g,b])     # switch it to rgb
    gray_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)
else:
    gray_img = bgr_img

img = cv2.medianBlur(gray_img, 5)
cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
#cimg=gray_img
circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,500,
                            param1=1,param2=3,minRadius=0,maxRadius=0)

circles = np.uint16(np.around(circles))

for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(cimg,(i[0],i[1]),i[2],(128,128,128),2)
    # draw the center of the circle
    #cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)

plt.subplot(121),plt.imshow(bgr_img)
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(cimg)
plt.title('Hough Transform'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
Hough_pred=[]
def Hough_transform1(predicts):
    for j in range(200):
        #img= predicts[j]
        bgr_img=predicts[j]
        if bgr_img.shape[-1] == 3:           # color image
            b,g,r = cv2.split(bgr_img)       # get b,g,r
            rgb_img = cv2.merge([r,g,b])     # switch it to rgb
            gray_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)
        else:
            gray_img = bgr_img

        img = cv2.medianBlur(gray_img, 3)
        cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
#cimg=gray_img
        circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,500,
                            param1=1,param2=3,minRadius=0,maxRadius=0)

        circles = np.uint16(np.around(circles))

        for i in circles[0,:]:
    # draw the outer circle
            cv2.circle(cimg,(i[0],i[1]),i[2],(128,128,128),2)
    # draw the center of the circle
    #cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)

        
        Hough_pred.append(cimg)
    return Hough_pred    

In [ ]:
predicts=model.predict(X_test)

In [ ]:
predicts[110].dtype

In [ ]:
X_test

# predicts=model.predict(X_test) aita diye predict korleocircle payoajasse thresholding kora lagtese na

In [ ]:
Hough_preds=Hough_transform1(predicts)

In [ ]:
plt.subplot(122),plt.imshow(predicts[4],cmap='gray')
plt.title('mask'), plt.xticks([]), plt.yticks([])
plt.show()
plt.subplot(122),plt.imshow(Hough_preds[0], cmap='gray')
plt.title('predict'), plt.xticks([]), plt.yticks([])
plt.show()
plt.subplot(122),plt.imshow(Hough_preds[4], cmap='gray')
plt.title('Hough Transform'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
len(Hough_pred)

In [ ]:
y_test.shape

In [ ]:
gray_img = cv2.cvtColor(Hough_preds[4], cv2.COLOR_BGR2GRAY)

In [ ]:
Hough_predlist=[]
for i in range(200):
    gray_img = cv2.cvtColor(Hough_preds[i], cv2.COLOR_BGR2GRAY)
    
    Hough_predlist.append(gray_img)


In [ ]:
 Hough_preds= np.array(Hough_predlist)

In [ ]:
Hough_preds.shape

In [ ]:
Hough_preds=np.expand_dims(Hough_preds, axis=3)

In [ ]:
y_test.shape

In [ ]:
Hough_preds=Hough_preds/256

In [ ]:
error2 = MSE2(y_test,Hough_preds)


In [ ]:
print("Image matching Error between the two images:",error2)

# END

In [ ]:
test_img_number=random.randint(0,len(X_test))
test_img=X_test[test_img_number]
ground_truth=y_test[test_img_number]
test_image_input=np.expand_dims(test_img,0)
predictio=(model.predict(test_image_input)[0,:,:,:]>.0841).astype(np.uint8)


#bgr_img=predictio

In [ ]:
bgr_img=predictio

In [ ]:
predictio.dtype

In [ ]:
ground_truth=ground_truth.astype("uint8")

In [ ]:
mse(ground_truth[:,:,0], predictio)

befor hough it should be converted


In [ ]:
predicts=predicts.astype('uint8')

In [ ]:
bgr_img=predicts[0]

In [ ]:
if bgr_img.shape[-1] == 3:           # color image
    b,g,r = cv2.split(bgr_img)       # get b,g,r
    rgb_img = cv2.merge([r,g,b])     # switch it to rgb
    gray_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)
else:
    gray_img = bgr_img

img = cv2.medianBlur(gray_img, 5)
cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
#cimg=gray_img
circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,500,
                            param1=1,param2=3,minRadius=0,maxRadius=0)

circles = np.uint16(np.around(circles))

for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(cimg,(i[0],i[1]),i[2],(128,128,128),2)
    # draw the center of the circle
    #cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)

plt.subplot(121),plt.imshow(bgr_img)
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(cimg)
plt.title('Hough Transform'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
mse()

In [ ]:
Hough_pred=[]
def Hough_transform1(predicts):
    for j in range(200):
        #img= predicts[j]
        bgr_img=predicts[j]
        if bgr_img.shape[-1] == 3:           # color image
            b,g,r = cv2.split(bgr_img)       # get b,g,r
            rgb_img = cv2.merge([r,g,b])     # switch it to rgb
            gray_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)
        else:
            gray_img = bgr_img

        img = cv2.medianBlur(gray_img, 3)
        cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
#cimg=gray_img
        circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,500,
                            param1=1,param2=3,minRadius=0,maxRadius=0)

        circles = np.uint16(np.around(circles))

        for i in circles[0,:]:
    # draw the outer circle
            cv2.circle(cimg,(i[0],i[1]),i[2],(128,128,128),2)
    # draw the center of the circle
    #cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)

        
        Hough_pred.append(cimg)
    return Hough_pred    

In [ ]:
predicts.shape

In [ ]:
len(predicts)

# problemm: loop e hough trasformation kaj kortese nah

In [ ]:
Hough_preds=Hough_transform1(predicts)

In [ ]:
plt.subplot(122),plt.imshow(predicts[74], cmap='gray')
plt.title('mask'), plt.xticks([]), plt.yticks([])
plt.show()
plt.subplot(122),plt.imshow(Hough_preds[7], cmap='gray')
plt.title('predict'), plt.xticks([]), plt.yticks([])
plt.show()
plt.subplot(122),plt.imshow(Hough_preds[0], cmap='gray')
plt.title('Hough Transform'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
len(Hough_pred)

In [ ]:
len(error)

In [ ]:
a=Hough_pred[4]
b=Hough_pred[2]

In [ ]:
a.dtype

In [ ]:
len()

In [ ]:
mse(a[:,:,0],b[:,:,0])

In [ ]:
error = MSE2(masks,Hough_pred)
print("Image matching Error between the two images:",error)



# prediction test dataset

In [ ]:
y_pred=model.predict(X_test)
y_pred_threshold=y_pred>0.5

# Determine MeanIoU

In [ ]:
from tensorflow.keras.metrics import MeanIoU


In [ ]:
n_classes=2
Iou_keras=MeanIoU(num_classes=n_classes)
Iou_keras.update_state(y_pred_threshold,y_test)
print("mean=",Iou_keras.result().numpy() )

In [ ]:
y_pred[7].shape

In [ ]:

plt.imshow(y_pred[7], cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()